# Introduction

Options are financial instruments that offer traders a versatile tool for hedging, speculating, and leveraging their portfolios. These instruments are a type of financial product known as **derivatives**, which grant the holder the right, though not the obligation, to buy or sell an underlying asset at a predetermined price before a specified expiration date. This unique structure allows traders to profit from market movements with relatively lower capital without purchasing the underlying asset, providing a `leverage effect` that amplifies returns.


There are several options strategies you can employ to benefit from market movements. In this notebook, we will explore `Long and Short Straddles`, a structure based on buying or selling two options—put and call—at the same strike price.

In [1]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 37.9 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 3.4 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.1-cp310-cp310-linux_x86_64.whl size=291653 sha256=a8191199b770bb42bc8010364262378a5489b0e531dd3fcdd181ff71a5362e05
  Stored in directory: /root/.cache/pip/wheels/d7/35/5c/1374782be033462df5f40174d8d879519d64ed8c25a1977554
Successfully built peewee


In [2]:
# Importing libraries

# Pandas & NumPy
import pandas as pd
import numpy as np

# Yfinance to retrieve financial data 
import yfinance as yf

# Plotly for Data Visualization
import plotly.express as px
import plotly.graph_objs as go
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.io as pio
from IPython.display import display
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# Explaining Options

As we know, options are a type of financial instrument known as `drivatives`, which means that the value of these instruments derives from the value of an underlying asset.

If you have stocks of a company, XYZ, you can `call options` and `put options` in the market. Call options give you the right to buy shares of XYZ stocks at a predetermined price before or on a specified date, while put optionsgive you the right to `sell shares` of XYZ will appreciate within a time frame, and you can buy puts when you bet that the shares of XYZ will `depreciate` within a time frame.

When dealing with options, there are some key components to keep in mind:

**Strike price**: This is the predetermined price at which the option holder can buy or sell the underlying asset. It is beneficial to hold calls with a strike price lower than the inderlying asset's price at the expiration date. The same is true for holding puts at a higher price than the underlying asset's price at the expiration date.

**Premium**: The cost of the option paid by the huber to he seller. Several factors influence the premium value. including volatility, time until expiration, and the difference between the strike price and the underlying asset's price.

**Expiration Date**: The specific date by which the option must be exercised or it becomes worthless.

# Annualized Volatility

Volatility plays a crucial role in options trading. It is a key input in options pricing models, such as the `Black-Scholes model`. Higher volatility increases the probability that the option will end up in the money, providing an advantageous deal for buying or selling the underlying asset on the expiration date. This increases the premium of options, allowing the trader to sell them at a higher price.


To keep track of volatility, I will compute the `annualized volatility` of assets by using the following formula:
`Annualized Volatility=𝜎×√252`

In the formula above,  𝜎  represents the standard deviation of returns over the last 20 trading days. We will then multiply this value by the  √252 to annualize the standard deviation of returns. This scaling factor transforms the standard deviation from a per-period basis to an annual basis, allowing for easier comparison and interpretation.

# Volatility Rank

After computing the annualized volatility, I will break down the volatility values into ten `equally-sized bins` based on their value distribution. I will then assign label values from 1 to 10 for each volatility instance according to which quantile bin it falls into. This process transforms continuous variables into categorical variables and simplifies analysis and visualization. We are going to call this variable `volatility rank`.

In the function defined below, `get_ticker`, we will compute the annualized volatility and volatility rank. Besides that, we will also plot the historical closing price, annualized volatility, and volatility rank.

In [3]:
# Defining function 
def get_ticker(ticker):

    """
    This function takes in a ticker from Yahoo Finance as argument. 
    It then computed the returns based on the adjusted closing prices and uses it to compute the annualized volatility.
    The annualized volatility is used to create an indicator called volatility rank, in which we categorize the annualized volatility into values from 1 to 10.
    """

    # Downloading historical data
    df = yf.download(ticker)

    # Computing returns
    df['Returns'] = df['Adj Close'].pct_change()

    # Computing annualized volatility
    df['Vol'] = df['Returns'].rolling(20).std() * np.sqrt(252)
    df.dropna(axis = 0, inplace = True) # Removing null values

    # Generating array of equally-spaced points representing the quantiles to be calculated (from 0% to 100%)
    quantiles = np.quantile(df['Vol'], np.linspace(0,1,11))

    # Using quantilies to categorize and label the annualized volatility values
    quantile_labels = pd.cut(df['Vol'], bins = quantiles, 
                             labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
    
    # Adding categorical labels to the dataframe
    df['Vol_Rank'] = quantile_labels

    # Plotting data
    fig = make_subplots(rows = 3, cols = 1, horizontal_spacing=0.2, row_heights=[.60, .20, .20], shared_xaxes=True)

    # First row --> Closing Price
    fig.add_trace(go.Scatter(
        x = df.index, y = df['Adj Close'], name = 'Closing Price'
    ), row = 1, col = 1)

    # Second row --> Annualized Volatility
    fig.add_trace(go.Scatter(
        x = df.index, y = df['Vol'], name = 'Annualized Volatility'
    ), row = 2, col = 1)

    # Third row --> Volatility rank
    fig.add_trace(go.Scatter(
        x = df.index, y = df['Vol_Rank'], name = 'Volatility Rank'
    ), row = 3, col = 1)

    # Defining subplots layout
    fig.update_layout(title = {'text': f'<b>{ticker} Closing Price & Volatility</b>'},
                      template = 'plotly_white',
                      height = 900, width = 950,
                      showlegend=False,
                      hovermode='x unified')
    
    # Defining layout of y-axes across subplots
    fig.update_yaxes(title_text = 'Closing Price ($)', row = 1)
    fig.update_yaxes(title_text = 'Annualized Volatility', row = 2)
    fig.update_yaxes(title_text = 'Volatility Rank', row = 3)

    fig.show() # Displaying plot

Let's use the function above to retrieve data of Apple (AAPL) shares. We need to input the symbol of the asset we would like to evaluate according to how they are on the Yahoo Finance website.

In [4]:
# Using function to extract hisorical data of Apple shares
get_ticker('AAPL')

/opt/conda/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning:

The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.

[*********************100%%**********************]  1 of 1 completed


The volatility rank indicator ranges from 1 to 10. Values closer to 1 represent lower volatility levels, while values closer to 10 represent higher levels.


A trader might expect more significant price movements when the volatility rank is closer to 10, which could favor strategies like long straddle. When the volatility rank is closer to 1, the trader might expect a more stable market, with lower fluctuation in prices, favoring strategies like short straddle.


The indicator also captures possible trend effects, allowing the trader to identify if volatility is increasing or decreasing in the last trading days.



# Long & Short Straddle

When studying options strategies, each has its own effects in limiting risk and maximizing returns. You can see terms such as long call, long put, iron condor, iron butterfly, etc.


Long and short straddle are two strategies that traders might use to capitalize on expectations of either `higher or lower volatility` and large or small price swings without having to predict the specific `direction` of price movement.


With `long straddle`, the trader buys two instruments: a call option and a put option of the `same underlying asset` and `same strike price and expiry date`. Long straddles are used in `high volatility` scenarios, where you expect large price movements but are not sure of the `direction`.


Long straddles offer `unlimited profit` and `limited risk`. When the underlying asset appreciates or depreciates, the trader profits. The maximum loss is limited to the net premium paid for both call and put options, plus operational costs. The trader might lose money with long straddles when the underlying asset stays within a price range by the expiration date of both options.

<div style="text-align: center"><img src="https://www.investopedia.com/thmb/rMz78ovHa7yCUIc0oHeL5SKx1R8=/750x0/filters:no_upscale():max_bytes(150000):strip_icc()/understandingstraddles22-19b55dd41aee458287dda61e4929428a.png" width="100%" heigh="100%" alt="Retrieve&Re-Rank pipeline"></div>

Image source: Investopedia


With `short straddle`, the trader sells two instruments: a call option and a put option of the `same underlying asset` and `same strike price and expiry date`. Short straddles are used in scenarios of `lower volatility`, where you expect none or very little price movements.


Short straddles offer `limited profit` and `unlimited risk`. When the underlying asset is closer to the strike price, the maximum profit is equal to the net premium paid for both options.

<div style="text-align: center"><img src="https://shortthestrike.com/wp-content/uploads/2023/08/short_straddle_risk_profile-1024x592.png?ezimgfmt=ng:webp/ngcb1" width="100%" heigh="100%" alt="Retrieve&Re-Rank pipeline"></div>

Image source: Shortthestrike


In the `get_strategy` function, we will perform the computations necessary to find the total payoff of long and short straddles and plot the payoff plots.

In [5]:
def get_strategy(ticker, strike_call, strike_put, premium_call, premium_put):

    """
    This function is used to visualize the payoffs of long and short straddle strategies of options trading.

    Params:
    ticker: Symbol of the underlying asset.
    strike_call: Strike price of the call option.
    strike_put: Strike price of the put option.
    premium_call: Premium of the call option.
    premium_put: Premium of the put option.
    """

    # Generating an array of possible prices the underlying asset might fall into
    stock_prices = np.linspace(0.79 * min(strike_put, strike_call), 1.19 * max(strike_put, strike_call), 100).round(2)

    # Computing payoffs
    # Payoffs for holding long call and put options 
    long_call_payoff = np.maximum(stock_prices - strike_call, 0) - premium_call
    long_put_payoff = np.maximum(strike_put - stock_prices, 0) - premium_put

    # Payoffs for holding short positions in call and put options
    short_call_payoff = -long_call_payoff
    short_put_payoff = -long_put_payoff

    # Combined payoffs for both long and short positions
    combined_long_payoff = long_call_payoff + long_put_payoff
    combined_short_payoff = short_call_payoff + short_put_payoff

    # Computing profit and loss for long straddle
    profit_long = np.maximum(combined_long_payoff, 0)
    loss_long = np.minimum(combined_long_payoff, 0)

    # Computing profit and loss for short straddle
    loss_short = np.minimum(combined_short_payoff, 0)
    profit_short = np.maximum(combined_short_payoff, 0)
    
    # Creating subplots
    fig = make_subplots(
        rows=2, cols=1,
        horizontal_spacing=0.2,
        row_heights=[0.5, 0.5],
        shared_xaxes=False,
        shared_yaxes=False
    )

    # Adding Long Straddle traces
    fig.add_trace(
        go.Scatter(x=stock_prices, y=long_call_payoff, mode='lines', line=dict(color='grey', dash='dash'), name='Long Call', hovertemplate='%{y:.2f}'),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=stock_prices, y=long_put_payoff, mode='lines', line=dict(color='grey', dash='dash'),  name='Long Put', hovertemplate='%{y:.2f}'),
        row=1, col=1
    )

    # Adding Short Straddle traces
    fig.add_trace(
        go.Scatter(x=stock_prices, y=short_call_payoff, mode='lines', line=dict(color='grey', dash='dash'),  name='Short Call', hovertemplate='%{y:.2f}'),
        row=2, col=1
    )
    fig.add_trace(
        go.Scatter(x=stock_prices, y=short_put_payoff, mode='lines', line=dict(color='grey', dash='dash'), name='Short Put', hovertemplate='%{y:.2f}'),
        row=2, col=1
    )
    

    # Adding the payoff lines for Long Straddle
    fig.add_trace(
        go.Scatter(x=stock_prices, y=profit_long, mode='lines', line=dict(color='black', dash='solid'), name='Profit', hovertemplate='%{y:.2f}',
                    fill='tozeroy', fillcolor='rgba(0, 255, 0, 0.5)'),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=stock_prices, y=loss_long, mode='lines', line=dict(color='black', dash='solid'), name='Loss', hovertemplate='%{y:.2f}',
                    fill='tozeroy', fillcolor='rgba(255, 0, 0, 0.5)'),
        row=1, col=1
    )

    # Adding the payoff lines for Short Straddle
    fig.add_trace(
        go.Scatter(x=stock_prices, y=loss_short, mode='lines', line=dict(color='black', dash='solid'), name = 'Loss', hovertemplate='%{y:.2f}',
                   fill='tozeroy', fillcolor='rgba(255, 0, 0, 0.5)'),
        row=2, col=1
    )
    fig.add_trace(
        go.Scatter(x=stock_prices, y=profit_short, mode='lines', line=dict(color='black', dash='solid'), name = 'Profit', hovertemplate='%{y:.2f}',
                   fill='tozeroy', fillcolor='rgba(0, 255, 0, 0.5)'),
        row=2, col=1
    )

    
    # Defining layout
    fig.update_layout(
        title={'text': f'<b>{ticker} Long & Short Straddle</b>'},
        template='plotly_white',
        height=1000, width=750,
        showlegend=False,
        hovermode='x unified')

    # Defining y-axes and x-axes tiles across rows
    fig.update_yaxes(title_text='Long Payoff ($)', row=1)
    fig.update_yaxes(title_text='Short Payoff ($)', row=2)
    fig.update_xaxes(title_text = 'Underlying Asset Price at Expiration', row=1)
    fig.update_xaxes(title_text = 'Underlying Asset Price at Expiration', row=2)

    fig.show() # Displaying plot

With the function above, we can compute payoffs for put and call options of Apple stocks. We start by inputting the ticker, "AAPL", and then the strike prices for the call and put options, which is U$`187.50`. We finish by inputting the premium for the call and put options respectively.


By using the function above, you may input different strike prices for both options. This would return the payoffs for a different options trading strategy called `Strangle`, which is very similar to straddles.

In [6]:
# Using function to obtain long and short straddle payoffs
get_strategy('AAPL', 187.50, 187.50, 2.99, 3.70)

The first plot shows the payoff for a long straddle strategy, where the trader profits as the underlying asset appreciates or depreciates further from the strike price. The second plot shows the payoff for a short straddle strategy, where the trader profits while the underlying asset is as close as possible to the strike price.

# Credit

https://www.kaggle.com/code/lusfernandotorres/options-trading-long-short-straddle/notebook